In [1]:
# imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime
from pathlib import Path

C:\Users\thebe\anaconda\envs\alpacaenv\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
# load env file
load_dotenv("C:\\Users\\thebe\\Fintech\\API_KEYS.env")

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = 'v2'
)

In [4]:
# initialize variables for api
ticker = "GLD"
timeframe = "1D"
current_date = datetime.datetime.now()
start_date = datetime.datetime(current_date.year-5, current_date.month, current_date.day)

In [5]:
# read in data
ticker_data = api.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=current_date,
    limit=1000
).df

ticker_data.head()

GLD                                       
                             open     high       low     close    volume
time                                                                    
2017-05-17 00:00:00-04:00  119.36  120.020  119.2700  119.8000  12182370
2017-05-18 00:00:00-04:00  119.77  119.840  118.5600  118.8200   8196025
2017-05-19 00:00:00-04:00  119.39  119.545  118.8800  119.4100   5632797
2017-05-22 00:00:00-04:00  119.63  120.140  119.5800  119.8953   3981207
2017-05-23 00:00:00-04:00  120.13  120.190  118.9841  119.1100   5993930

In [6]:
# get crypto data
btc_filepath = Path("bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
btc_data = pd.read_csv(btc_filepath)
btc_data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# clean data
btc_data = btc_data.dropna()
btc_data.isnull().sum()

Timestamp            0
Open                 0
High                 0
Low                  0
Close                0
Volume_(BTC)         0
Volume_(Currency)    0
Weighted_Price       0
dtype: int64

In [8]:
btc_data.Timestamp = pd.to_datetime(btc_data.Timestamp, unit='s')

# resampling to daily frequency
btc_data.index = btc_data.Timestamp
btc_data = btc_data.resample('D').mean()

In [9]:
btc_data.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2011-12-31,4.465000,4.482500,4.465000,4.482500,23.829470,106.330084,4.471603
2012-01-01,4.806667,4.806667,4.806667,4.806667,7.200667,35.259720,4.806667
2012-01-02,5.000000,5.000000,5.000000,5.000000,19.048000,95.240000,5.000000
2012-01-03,5.252500,5.252500,5.252500,5.252500,11.004660,58.100651,5.252500
2012-01-04,5.200000,5.223333,5.200000,5.223333,11.914807,63.119577,5.208159


In [15]:
btc_data.to_csv('cleaned_btc_data.csv')

In [10]:
# def sharpe ratio function

In [11]:
# def standard deviations function

In [12]:
# returns

In [13]:
# building visuals with sliders